### Import data

In [ ]:
using Pkg
Pkg.instantiate()
using DrugResponseModel

# import data from the path
conc_l, pop, g2, g1, g2_0, g1_0 = setup_data("lapatinib");# in which:
# pop: population data
# g1, g2: g1 and g2 data
# initial: initial number of cells in g1 and in g2 at time 0

### Estimate 

In [ ]:
# ODE optimization and estimation of the parameters
params_ode = zeros(4, 8)
for i in 1:8
    params_ode[:, i] = ODEoptimizer(zeros(4), 2.0*ones(4), [1.0, 1.0, 0.0, 0.0], i, g1, g2, g1_0, g2_0)
end

In [ ]:
p = params_ode

In [ ]:
using RecursiveArrayTools, DifferentialEquations

t = range(0.0; stop = 95.5, length = 192)
i=7
prob, sol = predict(p, g1_0, g2_0, i, t)

function remakeProblem(p, SaveAt)
  _prob = remake(prob;u0=convert.(eltype(p),prob.u0),p=p)
  solve(_prob,AutoTsit5(Rosenbrock23()), saveat=SaveAt;abstol=1e-6, reltol= 1e-6)
end

@model bayesODE(x, tp) = begin
    alpha ~ Uniform(0.0, 2.0)
    beta ~ Uniform(0.0, 2.0)
    gamma1 ~ Uniform(0.0, 2.0)
    gamma2 ~ Uniform(0.0, 2.0)

    p = [alpha, beta, gamma1, gamma2]
    sol_tmp = remakeProblem(p, tp)
    N = length(tp)

    fill_length = length(tp) - length(sol_tmp.u)

    print("\n", fill_length)
    for i in 1:fill_length
      if eltype(sol_tmp.u) <: Number
        push!(sol_tmp.u, Inf)
      else
        push!(sol_tmp.u, fill(Inf, size(sol_tmp[1])))
      end
    end

    for i in 1:N
      x[:,i] ~ MvNormal(sol_tmp.u[i], [0.5,0.5])
    end
end

In [ ]:
# plotting the fitted curves
ODEplot_all(params_ode, g1, g2, g1_0, g2_0, pop)